steps:
- preprocessing
- sentence similarity
- similarity matrix
- graph based ranking
- summary generation

In [31]:
text = """The Importance of Environmental Conservation The environment is a precious gift that we often take for granted. It provides us with clean air to breathe, fresh water to drink, and a habitat for countless species. However, in recent decades, human activities have taken a toll on our environment, leading to serious consequences such as climate change, loss of biodiversity, and pollution. This has made environmental conservation more critical than ever before.

1. Biodiversity Preservation

One of the primary reasons for environmental conservation is the preservation of biodiversity. Our planet is home to an incredible array of species, each playing a unique role in the ecosystem. When habitats are destroyed or polluted, many species face the threat of extinction. Protecting these species not only ensures their survival but also maintains the delicate balance of ecosystems upon which we depend for food, clean water, and a stable climate.

2. Climate Change Mitigation

The burning of fossil fuels, deforestation, and industrial processes release greenhouse gases into the atmosphere, leading to global warming and climate change. Environmental conservation efforts, such as reforestation, sustainable energy sources, and reduced emissions, play a crucial role in mitigating climate change. By protecting natural carbon sinks like forests and wetlands, we can slow down the rate of global warming and reduce its catastrophic effects.

3. Sustainable Resource Management

Natural resources like water, forests, and minerals are finite. Environmental conservation practices promote sustainable resource management, ensuring that we don't deplete these resources faster than they can regenerate. Sustainable practices not only benefit the environment but also support the long-term economic and social well-being of communities.

4. Clean Air and Water

Clean air and water are essential for human health and well-being. Pollution from industrial and agricultural sources can contaminate our air and water supplies, leading to serious health problems. Conservation efforts aim to reduce pollution and protect these vital resources, ensuring that future generations can enjoy a healthy and safe environment.

5. Cultural and Recreational Value

Natural landscapes hold immense cultural and recreational value. They provide opportunities for outdoor activities, tourism, and cultural practices. By conserving our natural heritage, we preserve the beauty and wonder of our planet for future generations to appreciate and enjoy.

6. Economic Benefits

Environmental conservation is not just about protecting nature; it's also about smart economics. Healthy ecosystems provide essential services, such as pollination, water purification, and soil fertility, which underpin agriculture and other industries. Conserving these services saves money and supports economic growth in the long run.

In conclusion, environmental conservation is not a luxury but a necessity. It's a responsibility that we all share to ensure a sustainable future for ourselves and the generations to come. By protecting our environment, we can safeguard the planet's biodiversity, mitigate climate change, and enjoy the countless benefits that nature provides. Every small effort counts, whether it's reducing personal carbon footprint, supporting conservation organizations, or advocating for sustainable policies. Together, we can make a difference and protect the world we call home."""

In [32]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance

import numpy as np
import networkx as nx

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/bhagya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bhagya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
def preprocess_text(text):
    sentences = nltk.sent_tokenize(text)
    word_freq = {}
    stop_words = set(stopwords.words('english'))
    for s in sentences:
        words = nltk.word_tokenize(s.lower())
        for word in stop_words:
            if word not in word_freq:
                word_freq[word] = 1
            else:
                word_freq[word]+=1
    max_freq = max(word_freq.values())
    for word in word_freq.keys():
        word_freq[word] = word_freq[word]/max_freq
    return sentences, word_freq

In [34]:
def similarity(sentence1, sentence2, word_freq):
    words1 = nltk.word_tokenize(sentence1)
    words2 = nltk.word_tokenize(sentence2)

    common_words = [word for word in words1 if word in words2]
    similarity_score = sum([word_freq.get(word, 0) for word in common_words])
    return similarity_score

In [35]:

def build_matrix(sentences, word_freq):
    num_sentences = len(sentences)
    similarity_matrix = np.zeros((num_sentences, num_sentences))
    for i in range(num_sentences):
        for j in range(num_sentences):
            if i != j:
                similarity_matrix[i][j] = similarity(sentences[i], sentences[j], word_freq)
    return similarity_matrix

In [36]:
def generate_summary(text, num_sentences):
    sentences, word_freq = preprocess_text(text)
    similarity_matrix = build_matrix(sentences, word_freq)
    nx_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(sentences)), reverse=True)
    summary = " ".join([sentence for score, sentence in ranked_sentences[:num_sentences]])
    return summary

In [37]:
generate_summary(text = text, num_sentences=5)

"Protecting these species not only ensures their survival but also maintains the delicate balance of ecosystems upon which we depend for food, clean water, and a stable climate. By conserving our natural heritage, we preserve the beauty and wonder of our planet for future generations to appreciate and enjoy. It's a responsibility that we all share to ensure a sustainable future for ourselves and the generations to come. However, in recent decades, human activities have taken a toll on our environment, leading to serious consequences such as climate change, loss of biodiversity, and pollution. By protecting natural carbon sinks like forests and wetlands, we can slow down the rate of global warming and reduce its catastrophic effects."